In [15]:
import os 
import pandas as pd
import numpy as np

# Define a Goal
Our Goal is to create a Network Intrusion Detection System (NIDS).
Categorize each object that is a raw packet as:
- Attack or Not Attack (label 1 or 0)
- Category of the Attack: Fuzzers, Analysis, Backdoors, DoS, Exploits, Generic, Reconnaissance, Shellcode and Worms.

# Data AcQuisition (DAQ)
We used a non cleaned dataset found on kaggle.com: **UNSW-NB15**. The raw packet was created by the *'IXIA PerfectStorm tool'*. This dataset is a labeled datset and in particular has nine types of attacks: 
- Generic: Broad category of general attacks.
- Fuzzers: Send random data to find vulnerabilities.
- Backdoors: Hidden access for unauthorized control.
- DoS: Overwhelm a system to disrupt service.
- Exploits: Use vulnerabilities for unauthorized access.
- Reconnaissance: Gather info to find vulnerabilities.
- Shellcode: Malicious code for system control.
- Worms: Self-spreading malware across networks.

First we assign a column name according to *NUSW-NB15_features.csv*.

In [16]:
column_name = [
    'srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'dur',
    'sbytes', 'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'service',
    'Sload', 'Dload', 'Spkts', 'Dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb',
    'smeansz', 'dmeansz', 'trans_depth', 'res_bdy_len', 'Sjit', 'Djit',
    'Stime', 'Ltime', 'Sintpkt', 'Dintpkt', 'tcprtt', 'synack', 'ackdat',
    'is_sm_ips_ports', 'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login',
    'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ltm',
    'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat',
    'Label'
]
df = pd.read_csv('UNSW-NB15_4.csv', header=None, names=column_name)

# Data Exploration
The exploration of the data is made in order to identify the dataset content, to extract and visualize data. We will use this in order to indentify possible features in the dataset that we do not want.

In [17]:
df.head(5)

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.9,7045,149.171.126.7,25,tcp,FIN,0.201886,37552,3380,31,...,,2,2,7,4,1,1,3,NaN,0
1,59.166.0.9,9685,149.171.126.2,80,tcp,FIN,5.864748,19410,1087890,31,...,,3,1,4,4,1,1,1,NaN,0
2,59.166.0.2,1421,149.171.126.4,53,udp,CON,0.001391,146,178,31,...,,3,5,2,7,1,1,4,NaN,0
3,59.166.0.2,21553,149.171.126.2,25,tcp,FIN,0.053948,37812,3380,31,...,,1,1,4,7,1,1,3,NaN,0
4,59.166.0.8,45212,149.171.126.4,53,udp,CON,0.000953,146,178,31,...,,2,5,2,1,1,1,2,NaN,0


We make a describe() only for usefull feature:
- Dur: Analyzes the duration of connections.
- Sbytes, Dbytes: Amount of bytes exchanged between source and destination.
- Sload, Dload: Data transmission speed.
- Spkts, Dpkts: Number of packets sent and received.
- Sjit, Djit: Jitter of the connection (variation in delay).
- Sintpkt, Dintpkt: Interval between packets.
- Tcprtt, Synack, Ackdat: TCP round-trip time, SYN-ACK, and acknowledgment time.

In [46]:
column_described = [
    'dur', 'sbytes', 'dbytes', 'Sload', 'Dload', 'sloss', 'dloss',
    'Spkts', 'Dpkts', 'Sjit', 'Djit',
    'Sintpkt', 'Dintpkt', 'tcprtt', 'synack', 'ackdat'
]

df[column_described].describe()


,dur,sbytes,dbytes,Sload,Dload,sloss,dloss,Spkts,Dpkts,Sjit,Djit,Sintpkt,Dintpkt,tcprtt,synack,ackdat
count,440044.000000,4.400440e+05,4.400440e+05,4.400440e+05,4.400440e+05,440044.000000,440044.000000,440044.000000,440044.000000,4.400440e+05,440044.000000,440044.000000,440044.000000,440044.000000,440044.000000,440044.000000
mean,0.580890,3.805922e+03,2.693348e+04,6.694344e+07,1.951188e+06,4.164484,12.246309,26.208541,32.320084,1.425334e+03,539.643135,127.166633,54.118351,0.008930,0.004706,0.004225
std,3.694473,6.526856e+04,1.378657e+05,1.531324e+08,3.915264e+06,25.464147,48.760595,73.360219,104.588023,1.711096e+04,2005.982783,2170.642473,1119.445521,0.068126,0.039393,0.035075
min,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000009,1.460000e+02,0.000000e+00,3.043453e+05,0.000000e+00,0.000000,0.000000,2.000000,0.000000,0.000000e+00,0.000000,0.008000,0.000000,0.000000,0.000000,0.000000
50%,0.004552,5.340000e+02,3.040000e+02,9.235256e+05,7.638522e+04,0.000000,0.000000,4.000000,4.000000,1.636482e+00,0.374054,0.323818,0.249588,0.000000,0.000000,0.000000
75%,0.089172,2.646000e+03,1.016800e+04,7.600000e+07,1.053726e+06,7.000000,8.000000,34.000000,30.000000,9.736690e+01,37.919519,2.475747,1.673626,0.000673,0.000536,0.000130
max,59.999989,1.106347e+07,1.283855e+07,5.600000e+09,2.290690e+07,4158.000000,4829.000000,8324.000000,9660.000000,1.201182e+06,120773.448400,84371.496000,52133.240000,10.037506,4.525272,5.512234


## Validating value 0 for the Duration of the packet
Value 0 in duration could be an error. let's compare it with other values relevant like *sbytes, dbytes, Sjit, Djit, sloss, or dloss*.

In [63]:
dur_zero = df[df['dur'] == 0]
dur_zero.count().to_frame().T

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,441,441,441,441,441,441,441,441,441,441,...,441,441,441,441,441,441,441,441,1,441


In [64]:
columns_of_interest = ['sbytes', 'dbytes', 'Sjit', 'Djit', 'sloss', 'dloss']
dur_zero[columns_of_interest].describe()

,sbytes,dbytes,Sjit,Djit,sloss,dloss
count,441.000000,441.0,441.000000,441.0,441.0,441.0
mean,50.678005,0.0,4.031095,0.0,0.0,0.0
std,70.226165,0.0,6.172316,0.0,0.0,0.0
min,28.000000,0.0,0.000000,0.0,0.0,0.0
25%,46.000000,0.0,0.000000,0.0,0.0,0.0
50%,46.000000,0.0,0.000000,0.0,0.0,0.0
75%,46.000000,0.0,10.954515,0.0,0.0,0.0
max,1504.000000,0.0,15.492038,0.0,0.0,0.0


Instead it seems that everything is ok because every packet with *dur* == 0 have a *sbytes* not zero and instead the *dbytes* are zero so no bytes are reaching the destination due to an error in the trasmission. 
No packet loss is encountered and we have only jitter for the source (*Sjit*) -> OK.

## Categoric Feature

In [45]:
column_1=[
    'proto', 'state', 'service', 'attack_cat', 'Label'
    ]

df[column_1].value_counts().to_frame().reset_index().rename(columns={0: 'count'}).head(10)

,proto,state,service,attack_cat,Label,count
0,udp,INT,dns,Generic,1,60344
1,tcp,FIN,http,Exploits,1,2966
2,tcp,FIN,-,Fuzzers,1,2825
3,unas,INT,-,Exploits,1,1800
4,unas,INT,-,DoS,1,1680
5,udp,INT,-,Fuzzers,1,1418
6,tcp,FIN,-,Exploits,1,1265
7,tcp,FIN,-,Reconnaissance,1,1180
8,udp,INT,-,Reconnaissance,1,1169
9,tcp,FIN,smtp,Exploits,1,1035
